In [1]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import time
import requests

from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "scraper"

In [2]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [3]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'Return answer for the question in POLISH language, based on provided article. Maximum length for the answer is 200 characters', 'input': 'https://tasks.aidevs.pl/text_pizza_history.txt', 'question': 'w którym roku według legendy została wynaleziona pizza Margherita?'}


In [4]:
# pobranie tekstu z linku
for i in range(20):
    try:
        response_article = requests.get(response['input'], timeout=50, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0'})
    except requests.exceptions.ConnectionError:
        print("Timeout, retrying...")
        time.sleep(2**i)
        continue
    if response_article.status_code == 200:
        print(response_article.content)
        break
    elif response_article.status_code >= 400:
        time.sleep(2**i)
else:
    raise Exception("Failed to fetch the page")


b'Foods similar to pizza have been made since the Neolithic Age. Records of people adding other ingredients to bread to make it more flavorful can be found throughout ancient history. In the 6th century BC, the Persian soldiers of the Achaemenid Empire during the rule of Darius the Great baked flatbreads with cheese and dates on top of their battle shields and the ancient Greeks supplemented their bread with oils, herbs, and cheese. An early reference to a pizza-like food occurs in the Aeneid, when Celaeno, queen of the Harpies, foretells that the Trojans would not find peace until they are forced by hunger to eat their tables (Book III). In Book VII, Aeneas and his men are served a meal that includes round cakes (like pita bread) topped with cooked vegetables. When they eat the bread, they realize that these are the "tables" prophesied by Celaeno.\nThe first mention of the word "pizza" comes from a notarial document written in Latin and dating to May 997 AD from Gaeta, demanding a pay

In [5]:
chat = ChatOpenAI(
    model = "gpt-4"
)

system = response["msg"]
question = response["question"]

content = chat.invoke([
    SystemMessage(f"{system}. ### Article: {response_article.content}"),
    HumanMessage(f"{question}"),
]).content

print(content)


Pizza Margherita została wynaleziona według legendy w 1889 roku.


In [6]:
# wysłanie odpowiedzi

response = send_answer(token, content)

if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
